In [1]:
import geopandas as gpd
import numpy as np

In [2]:
# Get geometry data from Natural Earth
url = "https://naciscdn.org/naturalearth/10m/cultural/ne_10m_admin_0_countries.zip"
data_v1 = gpd.read_file(url)
print(data_v1.head())

        featurecla  scalerank  LABELRANK SOVEREIGNT SOV_A3  ADM0_DIF  LEVEL  \
0  Admin-0 country          0          2  Indonesia    IDN         0      2   
1  Admin-0 country          0          3   Malaysia    MYS         0      2   
2  Admin-0 country          0          2      Chile    CHL         0      2   
3  Admin-0 country          0          3    Bolivia    BOL         0      2   
4  Admin-0 country          0          2       Peru    PER         0      2   

                TYPE TLC      ADMIN  ... FCLASS_TR  FCLASS_ID FCLASS_PL  \
0  Sovereign country   1  Indonesia  ...      None       None      None   
1  Sovereign country   1   Malaysia  ...      None       None      None   
2  Sovereign country   1      Chile  ...      None       None      None   
3  Sovereign country   1    Bolivia  ...      None       None      None   
4  Sovereign country   1       Peru  ...      None       None      None   

  FCLASS_GR  FCLASS_IT FCLASS_NL FCLASS_SE  FCLASS_BD FCLASS_UA  \
0      

In [3]:
# Set SRS=4326
data_v1 = data_v1.to_crs(epsg=4326)
print(data_v1.crs)

{'init': 'epsg:4326'}


c:\Users\risen\.conda\envs\new37\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [5]:
# Keep only full name, ISO_A2, geometry, populatio and GDP of the table
data_v2 = data_v1[['geometry', 'NAME_EN', 'ISO_A2_EH','POP_EST','GDP_MD']]
print(data_v2)

                                              geometry  \
0    MULTIPOLYGON (((117.70361 4.16341, 117.70361 4...   
1    MULTIPOLYGON (((117.70361 4.16341, 117.69711 4...   
2    MULTIPOLYGON (((-69.51009 -17.50659, -69.50611...   
3    POLYGON ((-69.51009 -17.50659, -69.51009 -17.5...   
4    MULTIPOLYGON (((-69.51009 -17.50659, -69.63832...   
..                                                 ...   
253  MULTIPOLYGON (((113.55860 22.16303, 113.56943 ...   
254  POLYGON ((123.59702 -12.42832, 123.59775 -12.4...   
255  POLYGON ((-79.98929 15.79495, -79.98782 15.796...   
256  POLYGON ((-78.63707 15.86209, -78.64041 15.864...   
257  POLYGON ((117.75389 15.15437, 117.75569 15.151...   

                         NAME_EN ISO_A2_EH      POP_EST   GDP_MD  
0                      Indonesia        ID  270625568.0  1119190  
1                       Malaysia        MY   31949777.0   364681  
2                          Chile        CL   18952038.0   282318  
3                        Bolivia   

In [6]:
# Delete the columns without proprt ISO_A2 values
data_v3 = data_v2[data_v2['ISO_A2_EH'] != '-99']
print(data_v3)

                                              geometry  \
0    MULTIPOLYGON (((117.70361 4.16341, 117.70361 4...   
1    MULTIPOLYGON (((117.70361 4.16341, 117.69711 4...   
2    MULTIPOLYGON (((-69.51009 -17.50659, -69.50611...   
3    POLYGON ((-69.51009 -17.50659, -69.51009 -17.5...   
4    MULTIPOLYGON (((-69.51009 -17.50659, -69.63832...   
..                                                 ...   
249  MULTIPOLYGON (((50.55161 26.19424, 50.59474 26...   
250  POLYGON ((154.39129 -21.03004, 154.38852 -21.0...   
252  POLYGON ((-109.21203 10.30268, -109.21036 10.2...   
253  MULTIPOLYGON (((113.55860 22.16303, 113.56943 ...   
254  POLYGON ((123.59702 -12.42832, 123.59775 -12.4...   

                         NAME_EN ISO_A2_EH      POP_EST   GDP_MD  
0                      Indonesia        ID  270625568.0  1119190  
1                       Malaysia        MY   31949777.0   364681  
2                          Chile        CL   18952038.0   282318  
3                        Bolivia   

In [7]:
# check validity of data
validity = data_v3.is_valid

if validity.all():
    print('All geometries are valid')
# if some geometry is not valid, correct the geometry by setting the buffer
else:
    print('At least one geometry is not valid')
    # get the index where validity is False
    invalid_row_idx = np.where(validity == False)[0]
    print('Invalid row index:', invalid_row_idx)
    # locate the invalid geometries and set the buffer
    selected_rows = data_v3.iloc[invalid_row_idx] 
    selected_rows['geometry'] = selected_rows['geometry'].buffer(0)
    data_v3.iloc[invalid_row_idx] = selected_rows
    # reexmaine the validity
    validity_v2=data_v3.is_valid
    if validity_v2.all():
        print('the invalid geometry has been corrected, and all geometries are valid')
    else:
        print('Still at least one geometry is not valid')

At least one geometry is not valid
Invalid row index: [158]


c:\Users\risen\.conda\envs\new37\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\risen\.conda\envs\new37\lib\site-packages\pandas\core\indexing.py:1717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


the invalid geometry has been corrected, and all geometries are valid


In [ ]:
# Export the data to geopackage
data_v3.to_file('Task-2-Data/Task-2-Exported-Geomeries.gpkg', layer='ISO_A2_Geometries', driver="GPKG")